In [10]:
import os
import pandas as pd
import numpy as np
from glob import glob
from xml.etree import ElementTree as et
from functools import reduce

In [11]:
#Extrac the xml file names from both train and test directories
path="C:/Users/KOTA SRI SURYA TEJA/Desktop/MLDL dataset/American-sign-language.v2i.voc"
train_xml_names=[] # For training set
test_xml_names=[] #For testing set
valid_xml_anmes=[]
for i in os.listdir(path):
    path_next=os.path.join(path+'/'+i)
    if os.path.isfile(path_next):
        continue
    if os.path.isdir(path+'/'+i):
        if i=='train':
            train_xml_names=[]
            for j in os.listdir(path+'/'+i):
                if j[-4:]=='.xml':
                    train_xml_names.append(j)
        if i=='test':
            test_xml_names=[]
            for j in os.listdir(path+'/'+i):
                if j[-4:]=='.xml':
                    test_xml_names.append(j)
        if i=='valid':
            valid_xml_names=[]
            for j in os.listdir(path+'/'+i):
                if j[-4:]=='.xml':
                    valid_xml_names.append(j)

In [12]:
test_xml_names[:10], train_xml_names[:10],valid_xml_names[:10]

(['-ePyJBmOJ9s_mp4-8_jpg.rf.a8b56428bbe8d2b80bc050fbbbc514c0.xml',
  '01831_augmented_darkness_mp4-11_jpg.rf.671ae9fc1c98e3cbc5c1c9bfda6b510c.xml',
  '01831_augmented_darkness_mp4-7_jpg.rf.609ce414c5f5952cdb957434f547c51b.xml',
  '01831_mp4-5_jpg.rf.a823f68cb36365cb747e7054e1a6d9af.xml',
  '01831_mp4-8_jpg.rf.7965574e4f96a9dcf40deee426ef6b54.xml',
  '01832_augmented_shifting_mp4-7_jpg.rf.4065049c052d8630e16552061b366d5d.xml',
  '01836_augmented_shifting_mp4-2_jpg.rf.402b86999ab98d9e1098993e12715bfd.xml',
  '1655257152_mp4-7_jpg.rf.b5e682c32c7e84493d961fbe1f79ed67.xml',
  '1655556212_augmented_brightness_mp4-1_jpg.rf.1d60452211d8dc7f54d5b82d14b4cd54.xml',
  '1655556212_augmented_speed_mp4-3_jpg.rf.74706e405b60f67360d6bdaa0b6d4667.xml'],
 ['-ePyJBmOJ9s_mp4-10_jpg.rf.1e6e42806e66f546dce7457edaf8b69e.xml',
  '-ePyJBmOJ9s_mp4-10_jpg.rf.c7a1c3c095774e0edfa0ed928d5e6e26.xml',
  '-ePyJBmOJ9s_mp4-5_jpg.rf.0044504d031d1d63c34661e08a91d231.xml',
  '-ePyJBmOJ9s_mp4-5_jpg.rf.83dacabc468debe7fd47b4e

In [13]:
# Extract details like filename, width, height, sign, Xmin, Xmax, Ymin, Ymax
def extract_from_xml(a,xml_names):
    data=[]
    for i in xml_names:
        base=et.parse(os.path.join(path+a+i))
        Ann=base.getroot()
        size = Ann.find('size')
        width = size.find('width').text
        height = size.find('height').text
        for j in Ann.findall('object'):
            name = j.find('name').text
            box = j.find('bndbox')
            xmin = box.find('xmin').text
            xmax = box.find('xmax').text
            ymin = box.find('ymin').text
            ymax = box.find('ymax').text
            data.append([i,width,height,name,xmin,xmax,ymin,ymax])
    return data

In [14]:
train_data=extract_from_xml('/train/',train_xml_names)
test_data=extract_from_xml('/test/',test_xml_names)
valid_data=extract_from_xml('/valid/',valid_xml_names)

In [15]:
train_data=pd.DataFrame(train_data,columns=['filename', 'width', 'height', 'sign', 'Xmin', 'Xmax', 'Ymin', 'Ymax'])
test_data=pd.DataFrame(test_data,columns=['filename', 'width', 'height', 'sign', 'Xmin', 'Xmax', 'Ymin', 'Ymax'])
valid_data=pd.DataFrame(valid_data,columns=['filename', 'width', 'height', 'sign', 'Xmin', 'Xmax', 'Ymin', 'Ymax'])

In [16]:
for i in list(train_data['sign'].unique()):
    if i not in list(valid_data['sign'].unique()):
        print(i)
print(len(train_data['sign'].unique()),len(test_data['sign'].unique()))

I
106 102


In [17]:
train_data['sign'].value_counts(),test_data['sign'].value_counts(),valid_data['sign'].value_counts()

(sign
 ingredients    565
 menu           547
 cup            441
 cold           403
 chicken        378
               ... 
 bye             65
 your            60
 receipt         59
 would           54
 wait            53
 Name: count, Length: 106, dtype: int64,
 sign
 menu           30
 ingredients    27
 straw          25
 cup            23
 cold           21
                ..
 pickle          2
 R               1
 H               1
 A               1
 B               1
 Name: count, Length: 102, dtype: int64,
 sign
 menu           55
 ingredients    51
 straw          40
 cold           40
 cup            40
                ..
 C               3
 Z               3
 A               2
 N               1
 X               1
 Name: count, Length: 105, dtype: int64)

In [18]:
# To train the machine Strings must be replace with numbers. So creating the dictionary.
num_to_name={}
name_to_num={}
c=1
for i in train_data['sign'].unique():
    if i not in num_to_name:
        num_to_name[c]=i
        name_to_num[i]=c
        c+=1

In [19]:
num_to_name[12],name_to_num['sorry'] # dict values from num to stringa and string to num
len(num_to_name)

106

In [20]:
def turn_sign_to_num(a):
    for i in range(len(a['sign'])):
        a['sign'].loc[i]=name_to_num[a['sign'].loc[i]]
    return a

In [21]:
#Turns the strings the num
train_data=turn_sign_to_num(train_data)
test_data=turn_sign_to_num(test_data)
valid_data=turn_sign_to_num(valid_data)

In [22]:
train_data.head(1)

,filename,width,height,sign,Xmin,Xmax,Ymin,Ymax
0,-ePyJBmOJ9s_mp4-10_jpg.rf.1e6e42806e66f546dce7...,640,640,1,248,372,343,502


In [49]:
def create_txt_for_img(data,k):
    for i in data['filename'].unique():
        f=open(os.path.join(path+'/'+k+'/'+i[:-4]+'.txt'), "w")
        a=data[data['filename']==i]
        a=a.astype({'width':int,'height':int,'sign':int,'Xmin':int,'Xmax':int,'Ymin':int,'Ymax':int})
        for j in a.values:
            id = j[3]
            X_c = ((j[5]+j[4])/2)/j[1]
            Y_c = ((j[7]+j[6])/2)/j[2]
            w = ((j[5]-j[4]))/j[1]
            h = ((j[7]-j[6]))/j[2]
            f.write('{0} {1} {2} {3} {4}\n'.format(id,X_c,Y_c,w,h))
        f.close()
    

In [50]:
create_txt_for_img(train_data,'train_images')
create_txt_for_img(test_data,'test_images')
create_txt_for_img(valid_data,'valid_images')

In [51]:
import shutil
def move_images(data,path,a,b):
    for i in data['filename'].unique():
        src = os.path.join(path+'/'+a+'/'+i[:-4]+'.jpg')
        des = os.path.join(path+'/'+b+'/'+i[:-4]+'.jpg')
        shutil.copyfile(src,des)

In [52]:
move_images(train_data,path,'train','train_images')
move_images(test_data,path,'test','test_images')
move_images(valid_data,path,'valid','valid_images')

In [53]:
a=[]
for i in range(1,107):
    a.append(num_to_name[i])
a

['chicken',
 'alcohol',
 'drink',
 'cost',
 'allergy',
 'bag',
 'bitter',
 'bread',
 'sweet',
 'manager',
 'french fries',
 'hello',
 'milk',
 'napkin',
 'pepper',
 'cup',
 'water',
 'warm',
 'A',
 'eggs',
 'additional',
 'cheese',
 'what',
 'bill',
 'sugar',
 'spicy',
 'ingredients',
 'pizza',
 'thank-you',
 'burger',
 'enjoy',
 'menu',
 'bacon',
 'straw',
 'drive',
 'yoghurt',
 'order',
 'hot',
 'coupon',
 'urgent',
 'tissues',
 'lid',
 'coke',
 'sauce',
 'refill',
 'dessert',
 'cold',
 'soda',
 'juicy',
 'lactose',
 'fresh',
 'repeat',
 'credit card',
 'total',
 'cash',
 'mustard',
 'receipt',
 'please',
 'your',
 'sandwich',
 'barbecue',
 'biscuit',
 'tomato',
 'would',
 'ready',
 'bye',
 'ketchup',
 'eat',
 'safe',
 'cake',
 'sorry',
 'pickle',
 'fork',
 'spoon',
 'lettuce',
 'wait',
 'icecream',
 'vegetables',
 'B',
 'C',
 'D',
 'no',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'salt',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'small',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 '

In [53]:
test_data.to_csv('Test_output.csv')